# Neo4j Hello World (Notebook)

This notebook connects to a local Neo4j **Community** instance (via Docker), creates a tiny graph, and queries it into a pandas DataFrame.

**Assumes** Neo4j is running at `bolt://localhost:7687` with `neo4j/test1234`.


In [1]:
# Optional: install dependencies if running outside the repo venv
# !pip install neo4j pandas
from neo4j import GraphDatabase
import pandas as pd

URI = "bolt://localhost:7687"
AUTH = ("neo4j", "test1234")
driver = GraphDatabase.driver(URI, auth=AUTH)
driver


In [2]:
def create_graph(tx):
    tx.run(
        """
        MERGE (a:Person {name: "Andreas"})
        MERGE (b:Person {name: "Andrew"})
        MERGE (c:Course {title: "RAG with Knowledge Graphs"})
        MERGE (a)-[:KNOWS {since: 2024}]->(b)
        MERGE (a)-[:TEACHES {year: 2024}]->(c)
        MERGE (b)-[:INTRODUCED {section: "Intro"}]->(c)
        """
    )

with driver.session() as session:
    session.execute_write(create_graph)
print("Graph created/merged ✔️")


Graph created/merged ✔️


In [3]:
# Simple MATCH returning rows
query = (
    "MATCH (a:Person)-[r:KNOWS]->(b:Person) "
    "RETURN a.name AS person, r.since AS since, b.name AS knows"
)
with driver.session() as session:
    records = session.run(query)
    rows = [r.data() for r in records]
pd.DataFrame(rows)


,person,since,knows
0,Andreas,2024,Andrew
1,Adalberto,2025,Iria
2,Iria,2025,Guillermo
3,Guillermo,2025,Adalberto


In [4]:
# Parameterized query example
person_name = "Andreas"
param_query = (
    "MATCH (p:Person {name: $name})-[:TEACHES]->(c:Course) "
    "RETURN p.name AS teacher, c.title AS course"
)
with driver.session() as session:
    rows = [r.data() for r in session.run(param_query, name=person_name)]
pd.DataFrame(rows)


,teacher,course
0,Andreas,RAG with Knowledge Graphs


In [6]:
# Optional cleanup: wipe all data in the DB (use with care!)
wipe = True  # set True to delete everything
if wipe:
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")
    print("Database cleared.")
else:
    print("Cleanup skipped.")


Database cleared.


In [7]:
driver.close()
print("Driver closed.")


Driver closed.
